## Separate Total Population dataset by States

In [1]:
import cudf
import pickle

In [2]:
# Load the dataset
df = cudf.read_parquet('../data/total_population_dataset.parquet')
df.head()

,easting,northing,race,net,county
0,-9626792.0,3825189.75,1,0,0
1,-9626832.0,3825073.75,1,0,0
2,-9627101.0,3825153.50,1,0,0
3,-9627149.0,3825322.75,1,0,0
4,-9627159.0,3825334.75,1,0,0


In [3]:
# Load the state to county mapping
id2county = pickle.load(open('../id2county.pkl','rb'))
df_counties = cudf.DataFrame(dict(idx=list(id2county.keys()), county=list(id2county.values())))

# Lowercase the county names for easier merging
df_counties['county_lower'] = df_counties.county.str.lower()
df_counties.head()

,idx,county,county_lower
0,0,Autauga County,autauga county
1,1,Baldwin County,baldwin county
2,2,Barbour County,barbour county
3,3,Bibb County,bibb county
4,4,Blount County,blount county


In [4]:
# Dataset downloaded from https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code&disjunctive.ste_name&disjunctive.coty_code&disjunctive.coty_name
county_state_df = cudf.read_csv('../data/us-counties1.csv', delimiter=";")[['Official Name County', 'Type', 'Official Name State']].dropna()
county_state_df.columns = ['county', 'type', 'state']
county_state_df.head()

,county,type,state
0,Harrison,county,Missouri
1,Jefferson,county,Missouri
2,Newton,county,Missouri
3,Wayne,county,Missouri
4,Lincoln,county,Montana


In [5]:
# Add the type to the county name
county_state_df['county'] = county_state_df.apply(lambda row: row['county'] + ' ' + row['type'], axis=1)

# Remove non-ascii characters and abbreviations to match the other id2county mapping dataset
county_state_df['county'] = county_state_df.county.to_pandas().replace({r'[^\x00-\x7F]+': '', r'([A-Z][a-z]+)([A-Z]+)': r'\1'}, regex=True)

# Lowercase the county names for easier merging
county_state_df['county_lower'] = county_state_df['county'].str.lower()

In [6]:
# Merge the datasets and drop duplicates to get the state for each county in the total population dataset
df_map_county_to_states = df_counties.merge(county_state_df, on='county_lower', how='left', suffixes=['', '_y']).drop_duplicates(subset=['county_lower'])[['idx', 'county', 'state' ]]

In [9]:
# Fill in the states for unavailable states manually by looking at the counties
# Carson City, Nevada
# District of Columbia, Washington DC
# Remaining, Connecticut
df_map_county_to_states.loc[df_map_county_to_states.county == 'Carson City', 'state'] = 'Nevada'
df_map_county_to_states.loc[df_map_county_to_states.county == 'District of Columbia', 'state'] = 'Nevada'
df_map_county_to_states.loc[df_map_county_to_states.isna().any(axis=1), 'state'] = 'Connecticut'

In [10]:
# Save the mapping
df_map_county_to_states.to_parquet('../data/county_to_state_mapping.parquet')

In [11]:
df_map_county_to_states

,idx,county,state
0,144,Lonoke County,Arkansas
1,145,Miller County,Georgia
3,146,Mississippi County,Missouri
5,147,Nevada County,California
7,148,Newton County,Texas
...,...,...,...
2954,76,Fairbanks North Star Borough,Alaska
2955,78,Hoonah-Angoon Census Area,Alaska
2956,79,Juneau City and Borough,Alaska
2958,74,Denali Borough,Alaska
